In [241]:
from google import genai
from google.genai import types
import base64
import json
import os
from dotenv import load_dotenv

import unicodedata
import re

In [242]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [243]:
start_at = 13
end_at = 13

In [244]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [245]:
poetic_translation_prompt = "Translate the following text from Japanese to Chinese. The original text is a poem. Try to make the translation poetic but keep the original meanings. Display in Traditional Chinese."

descriptive_translation_prompt = "Translate the following text from Japanese to Chinese. Keep the original meanings. Display in Traditional Chinese."

In [246]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [247]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [248]:
# Read the markdown template for writing the Chinese translations to Markdown file

#read in the template
with open("templates/raw_output_md_template", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [249]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [ ]:
# process the translation in batch mode

for song_no in range(start_at, end_at + 1):
    with open(f"Chiri_Japanese_Translation/story_translation_{song_no}.txt", "r", encoding="utf8") as f:
        japanese_story = f.read()
        japanese_story = unicodedata.normalize('NFKC', japanese_story)

        poetic_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=poetic_translation_prompt)

        descriptive_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=descriptive_translation_prompt)

        md_output = md_template.format(translated_language="Chinese", ainu_title=ainu_titles[song_no - 1],
                               poetic_prompt=poetic_translation_prompt, descriptive_prompt=descriptive_translation_prompt,
                               japanese_title=japanese_titles[song_no], input_japanese = japanese_story,
                               output_poetic=poetic_translation, output_descriptive=descriptive_translation)
        
        md_name_part = get_output_file_name_key(ainu_titles[song_no - 1])

        with open(f"LLM_prompts_and_raw_translations_main_content/Chinese_Translation/{song_no}_{md_name_part}_to_Chinese.md", "w", encoding="utf8") as f:
            f.write(md_output)



好的，這是一段將日文詩歌翻譯成繁體中文的嘗試，我會盡力在保留原意的同時，兼顧詩意：

**沼貝自歌謠「托努佩卡 蘭蘭」**

托努佩卡 蘭蘭

烈日炎炎，灼我身旁，
乾涸將盡，命不久長。
「誰人賜水，解我乾渴，
救我脫離，此般苦厄。
水啊水啊！」哀聲泣訴，
遙見海濱，一女緩步。
背負竹簍，向我走來，
無視哀鳴，漠然走開。
見我可憐，
「可憎沼貝，何苦喧嘩，
擾人清靜，徒增笑話。」
言罷足踏，碾碎貝殼，
拋擲遠方，揚長而去。
「痛哉苦哉，水啊水啊！」
哀號未絕，又見一女，
背負竹簍，自海濱來。
我等呼號：「誰人慈悲，
賜我甘霖，救我將死。
痛哉苦哉，水啊水啊！」
此女若神，容顏清麗，
見我慘狀，駐足嘆息：
「可憐沼貝，酷暑難耐，
棲身之所，乾涸龜裂，
渴求甘霖，何故遭難，
莫非受人，踐踏欺凌？」
輕柔拾起，聚攏一處，
置於蕗葉，捧入清湖。
清冽湖水，滋潤身軀，
重獲生機，恢復元氣。
方知二女，性情迥異，
先來之女，狠心踐踏，
乃是薩瑪雲庫爾之妹，
後至之女，憐憫相助，
乃是奧奇奇里姆伊之妹。
薩瑪雲庫爾之妹，心懷惡念，
致使粟田，枯萎凋零。
奧奇奇里姆伊之妹，心懷善念，
使其粟田，豐收盈倉。
是年，奧奇奇里姆伊之妹，
收穫頗豐，皆因我故。
遂以沼貝之殼，摘取粟穗。
自此之後，人間女子，
採摘粟穗，皆用貝殼。

一沼貝如是說。
以下是將日文文本翻譯成繁體中文的結果：

沼貝自唱的歌謠「托努佩卡 蘭蘭」

托努佩卡 蘭蘭
強烈的日光照射，我的棲身之處也
乾涸了，我快要死了。
「誰來給我水喝，
救救我吧。水啊水啊」我們哭喊著，
從遠方的海灘走來一位女子，
背著竹簍。
我們哭泣著，她從我們身旁經過，
看見我們，
說：「可笑的沼貝，可惡的沼貝，哭什麼
吵吵鬧鬧的。」
她踩踏我們，用腳尖踢開，連同貝殼一起踩碎，
然後就往山裡去了。
「好痛，好難受，水啊水啊。」我們哭喊著，
從遠方的海灘又走來一位女子，
背著竹簍。我們
「誰來給我們水喝，救救我們吧，
好痛，好難受，水啊水啊。」哭喊著。
於是，那位姑娘，神一般美麗高貴，
來到我們身邊，看見我們，
說：「真可憐，天氣這麼熱，沼貝們的
棲身之處都乾涸了，想要喝水，
這是怎麼了呢？
好像是被踩踏過一樣……」
她一邊說著，一邊把我們都撿起來，
放在蕗葉上，放進了清澈的湖裡。
我們喝了清澈的冷水，完全恢復了元氣，
變得非常健康。於是，我們開始